In [ ]:
!pip install tensorflow_recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import keras
from datetime import datetime
from sklearn import preprocessing

In [ ]:
# Read dataset dataset_kuesioner
data = pd.read_csv(r'dataset_kuesioner.csv', sep=",", encoding='utf8', engine="python")

# Drop missing value
data.dropna(inplace=True)

# Preprocessing category
data.rename(columns={'category 1': 'category_1', 'category 2': 'category_2'}, inplace=True)
le = preprocessing.LabelEncoder()
data['category_1'] = le.fit_transform(data['category_1'])
data['category_2'] = le.fit_transform(data['category_2'])

In [ ]:
data

,number,survey,category_1,category_2
0,1,Penilaian Terhadap Peran Bisnis dalam Menjaga ...,1,0
1,2,Tinjauan terhadap Dampak Faktor Keuangan terha...,4,3
2,3,Penilaian Tingkat Literasi Keuangan di Kalanga...,4,4
3,4,Pengaruh Pengetahuan tentang Sistem Transporta...,8,14
4,5,Persepsi Masyarakat tentang Kebutuhan dan Tant...,5,14
...,...,...,...,...
1186,1187,Studi Tentang Tantangan dan Inovasi dalam Peni...,10,9
1187,1188,Persepsi Masyarakat terhadap Kontribusi Transp...,14,11
1188,1189,Persepsi Stakeholder terhadap Efektivitas Sist...,9,2
1189,1190,Persepsi Masyarakat Terhadap Penerapan Prinsip...,0,5


In [ ]:
# Read dataset dataset_dummy
data2 = pd.read_csv(r'dataset_dummy.csv')

In [ ]:
# Data preprocessing
data2['user_id'] = data2['user_id'].astype(str)
data2['category'] = data2['category'].astype(str)
data2['history'] = data2['history'].astype(np.float32)

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((
    tf.cast(data2['user_id'].values.reshape(-1, 1), tf.string),
    tf.cast(data2['category'].values.reshape(-1, 1), tf.string),
    tf.cast(data2['history'].values.reshape(-1, 1), tf.float32)
))

In [ ]:
@tf.function
def rename(x0, x1, x2):
    y = {}
    y["user_id"] = x0
    y['category'] = x1
    y['history'] = x2
    return y


dataset = dataset.map(rename)

# Retrieve unique category and user_id
category = data2.category.values
userid = data2.user_id.values

unique_category = np.unique(list(category))
unique_userid = np.unique(list(userid))

In [ ]:
class RankingModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 10

        # Compute embeddings for users
        self.userid_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_userid, mask_token=None),
            tf.keras.layers.Embedding(len(unique_userid) + 1, embedding_dimension)
        ])

        # Compute embeddings for category
        self.category_embeddings = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_category, mask_token=None),
            tf.keras.layers.Embedding(len(unique_category) + 1, embedding_dimension)
        ])

        # Compute predictions
        self.ratings = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(1)
        ])


    def call(self, inputs):
        user_id, category = inputs
        userid_embedding = self.userid_embeddings(user_id)
        category_embedding = self.category_embeddings(category)

        return self.ratings(tf.concat([userid_embedding, category_embedding], axis=1))

In [ ]:
class userModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False) -> tf.Tensor:
        user_id_embedded = self.ranking_model.userid_embeddings(features["user_id"])
        category_embedded = self.ranking_model.category_embeddings(features["category"])

        mission_predictions = self.ranking_model.ratings(tf.concat([user_id_embedded, category_embedded], axis=1))

        # Compute the loss and metrics
        return self.task(labels=features["history"], predictions=mission_predictions)


In [ ]:
model = userModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
# Cache the dataset
cache_dataset = dataset.cache()

In [ ]:
# Tensorboard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Training
model.fit(cache_dataset, epochs=300, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
eval_dataset = dataset.shuffle(len(data2)).batch(1)
eval_results = model.evaluate(eval_dataset, return_dict=True)
print('Evaluation results:', eval_results)

Epoch 1/300
1500/1500 [==============================] - 7s 4ms/step - root_mean_squared_error: 3.0392 - loss: 9.2307 - regularization_loss: 0.0000e+00 - total_loss: 9.2307
Epoch 2/300
1500/1500 [==============================] - 4s 3ms/step - root_mean_squared_error: 2.9344 - loss: 8.6053 - regularization_loss: 0.0000e+00 - total_loss: 8.6053
Epoch 3/300
1500/1500 [==============================] - 5s 3ms/step - root_mean_squared_error: 2.9002 - loss: 8.4056 - regularization_loss: 0.0000e+00 - total_loss: 8.4056
Epoch 4/300
1500/1500 [==============================] - 4s 3ms/step - root_mean_squared_error: 2.8561 - loss: 8.1521 - regularization_loss: 0.0000e+00 - total_loss: 8.1521
Epoch 5/300
1500/1500 [==============================] - 5s 3ms/step - root_mean_squared_error: 2.8070 - loss: 7.8742 - regularization_loss: 0.0000e+00 - total_loss: 7.8742
Epoch 6/300
1500/1500 [==============================] - 5s 3ms/step - root_mean_squared_error: 2.7614 - loss: 7.6205 - regularization_

In [ ]:
# Create array with users id in every place on the length of the unique category number
user8 = np.array([["8"] for i in range(len(unique_category))])

# Convert it to tf.data.Dataset
test_data = tf.data.Dataset.from_tensor_slices((tf.cast(user8.reshape(-1, 1), tf.string),
                                                tf.cast(unique_category.reshape(-1, 1), tf.string)))

# Name the columns
@tf.function
def rename_test(x0, x1):
    y = {}
    y["user_id"] = x0
    y['category'] = x1
    return y


test_data = test_data.map(rename_test)

# Make predictions and store them in a dictionary
test_mission = {}
for b in test_data:
    user_id_embedded = model.ranking_model.userid_embeddings(b['user_id'])
    category_embedded = model.ranking_model.category_embeddings(b['category'])
    prediction = model.ranking_model.ratings(tf.concat([user_id_embedded, category_embedded], axis=1))

    test_mission[b['category'].numpy()[0]] = prediction.numpy()[0][0]

# Sort the predictions by score and print the titles
count = 0
for b in sorted(test_mission, key=test_mission.get, reverse=True):
    if count < 3:
        print(b)
        count += 1
    else:
        break

b'Bisnis dan Industri'
b'Pertanian dan Perikanan'
b'Hukum'


In [ ]:
# Save model
tf.saved_model.save(model, "saved_model")
